In [1]:
# screenshot ; shopping cart state diagram
# Design patterns covered:

In [5]:
# 1) Facade pattern:
# Implement a common interface to access a varity of APIs that have similar goals, but different representations.
# programs using facade need not be aware of the particulars how they talk to the underlying API.
# eg build a common API to access different databases from different manufacturers

# 2) Adaptor pattern
# used to convert a new related interface into ones the client's expect.
# using adaptor you won't have to change the client code to add new functionality
# eg) add vendor class to existing customer class with similar but NOT the same attributes.

# 3) Deccorator pattern
# add functionality without breaking the open closed principle
# application that builds car objects, now add the ability to change engine or wheel size.
# decorator can help with that and favours composition over inheritance.

# 4) Template pattern
# defines the basic outline of an algorithm and let's sub classes implement one or more steps while preserving the order of the steps.

# 5) Iterator pattern
# encapsulates the collection of objects to make iteration easy and hide the collection's implementation.

# 6) Composite pattern
# makes it easy to compose objects in tree structures,
# so that clients can handle individual objects and sub trees through a common interface.
# eg) family tree or corporate hierarchy

# 7) State patterm
# follows mathematical concept of state machine.
# behaviour changes according to the state of the object.
# website with shopping cart can change it's behavious if the cart is empty or not or items have changed.

# 8) Proxy pattern
# used to control access to another object.
# perhaps for filtering or security reasons.
# eg) weather application and for 1 client you only want to return sunny days.

## 1) Facade Pattern

In [1]:
# Hiding complexity with facade pattern ie puts a new face on something, an API in this case
# it's handy when you have a complex api or a collection of apis and you want to simply them for your use case.
# it's also useful when you have several APIs that do similar things, but in different ways
# and you want to have just one API to use in client code.
# eg) access a database -> to get employee records
# ask DBA which databse to use and then import the right python modules.
# instatiate a control object
# build a connection string
# connect to database
# run a query and process the results.
# close the connection

In [5]:
# working example ; Retrieve and print employee records

# get_employees.py
import pyodbc


# ideally read from config file or passed in as param
CONNSTR = \
'DRIVER={SQL Server};SERVER=mhknbn2kdz.database.windows.net;DATABASE=AdventureWorks2012;UID=sqlfamily;PWD=sqlf@mily'

def get_employees():
    connection = pyodbc.connect(CONNSTR)
    query ="""
        SELECT DISTINCT TOP 5 FirstName,LastName
        From Person.Person
        ORDER BY LastName,FirstName;
    """
    cursor = connection.cursor()
    cursor.execute(query)
    for row in cursor:
        print(row.FirstName,row.LastName)
    connection.commit()
    connection.close()

ModuleNotFoundError: No module named 'pyodbc'

In [6]:
get_employees()

NameError: name 'get_employees' is not defined

In [7]:
# problems
# 1) most of it is boiler plate required by pyodbc to set up and run the query rather than the query itself
# this will result in redundant boiler plate code for each developer
# 2) additional pyodbc methods imported which are not required.
# 3) changing database or Changing module (ie pyodbc) will require changes in codebase of each developer individually

In [8]:
# Solution : The FACADE PATTERN

# Facade is a structural pattern that means it provides a new way to put a program together
# Used to present a unified interface to a set of interfaces(these interfaces need not be related to each other in any way).
# Facade is a higher level interface and makes the set of underlying interfaces easier to use
# and reduces complexity
# Our example has five interfaces:
# connect , cursor , execute , commit and close
# Using facade pattern will result in just one API which hides all the above 5 details.
# Facade provides a way to talk to all the lower level interfaces through a higher level interface.
# Facade knows which lower level interface(sub class) is responsible for our request and
# navigates client request accordingly.
# the lower level interface then handle the request made by facade class to achieve the results requested by client program.

In [9]:
# Facade Structure :
# Screenshot ; UML diagram

# Abstract Facade with simple_api() method in our example it's get_employee()

# Concrete Facade1
# Concrete Facade2
 

# Facade Factory with create_facade() method which returns concrete facade
# client program calls facade factory to get an instantiated facade to use 
# the factory will import the desired concrete facade 
# the factory returns an instantiated facade to the client
# then the client api will use the simple_api() method to do the work

In [11]:
# Folder structure

# FACADE
#    -> __main__.py
#    -> get_employees
#           -> __init__.py
#           -> abs_facade.py
#           -> facade_factory.py
#           -> sql_server.py

In [14]:
# __init__.py

# holds 3 constant definitions 
# provider tells you which module provides concrete implementation of facade
# provides flexibilty to change query/database etc directly

PROVIDER="sql_server"

CONNSTR = (
    """DRIVER={SQL Server};
    SERVER=mhknbn2kdz.database.windows.net;
    DATABASE=AdventureWorks2012;
    UID=sqlfamily;PWD=sqlf@mily"""
)

QUERY = """
        SELECT DISTINCT TOP 5 FirstName,LastName
        From Person.Person
        ORDER BY LastName,FirstName;
        """


# abs_facade.py
from abc import ABCMeta,abstractmethod

class AbsFacade(metaclass=ABCMeta):
    
    @abstractmethod
    def get_employees(self):
        pass
  

# sql_server.py
import pydbc
# from .abs_facade import AbsFacade
# from . import CONNSTR,QUERY

class GetEmployeesFacade(AbsFacade):
    def get_employees(self):
        connection = pyodbc.connect(CONNSTR)
        cursor = connection.cursor()
        cursor.execute(QUERY)
        for row in cursor:
            print(row.FirstName,row.LastName)
        connection.commit()
        connection.close()
    
    
# facade_factory
class FacadeFactory:
    
    @staticmethod
    def create_facade(module_name):
        module = import_module('.' + module_name + __package__)
        
        classes = getmembers(module, lambda m: (isclass(m) and not isabstract(m) and issubclass(m.AbsFacade)))
        
        return classes[0][1]()
    
    
# __main__.py
# from get_employees import PROVIDER
# from get_employees.facade_factory import FacadeFactory

def main():
    facade = FacadeFactory.create_facade(PROVIDER)
    facade.get_employees()
    

main()

ModuleNotFoundError: No module named 'pydbc'

## 2) ADAPTER PATTERN

In [16]:
# Make one api look like another one that has different methods and signatures.
# The purpose it to provide the client an abstract api to program against, rather than a number of different APIs that have similar goals.
# Favour coding to abstractions instead of concrete classes, also called DEPENDENCY INVERSION PRINCIPLE.
# Adapters in real life:

# plumbers use adapters to change bw(connect) different sorts/sizes of pipes 
# Charging adapters; different sockets

# Working Example
# Print customer name and address from customer object/class.
# Now you have to make it work with Vendor object/class as well.
# but the vendor api is different than the customer api
# Customer object has combined address property (with number and street)
# whereas Vendor object has separate number and street properties.

In [18]:
# solution1

# customer.py
class Customer:
    
    def __init__(self,name,address):
        self._name = name
        self._address = address
        
    @property
    def name(self):
        return self._name
    
    @property
    def address(self):
        return self._address
    

# mock_cutomers.py

# from customer import Customer
MOCKCUSTOMERS = (
    Customer('Pizza love','33 Pepperoni Lane'),
    Customer('Happy and Green','25 Kale St.'),
    Customer('Sweeth tooth','42 Chocolate Ave.')
)



# __main__.py

# from mock_customers import MOCKCUSTOMERS
def main():
    for cust in MOCKCUSTOMERS:
        print("Name : {} , Address : {}".format(cust.name, cust.address))


main()

Name : Pizza love , Address : 33 Pepperoni Lane
Name : Happy and Green , Address : 25 Kale St.
Name : Sweeth tooth , Address : 42 Chocolate Ave.


In [21]:
# now add support for vendors

# vendor.py
class Vendor:
    
    def __init__(self,name,number,street):
        self._name = name
        self._number = number
        self._street = street 
        
    @property
    def name(self):
        return self._name
    
    @property
    def number(self):
        return self._number
    
    @property
    def street(self):
        return self._street
    

# mock_vendors.py
# from vendor import Vendor
MOCKVENDORS = (
    Vendor('Dough factory','1','Semolina Court'),
    Vendor('Farm produce','14','Country Rd.'),
    Vendor('Cocoa World','53','Tropical Blvd.')
)


# customer.py
class Customer:
    
    def __init__(self,name,address):
        self._name = name
        self._address = address
        
    @property
    def name(self):
        return self._name
    
    @property
    def address(self):
        return self._address
    

# mock_cutomers.py

# from customer import Customer
MOCKCUSTOMERS = (
    Customer('Pizza love','33 Pepperoni Lane'),
    Customer('Happy and Green','25 Kale St.'),
    Customer('Sweeth tooth','42 Chocolate Ave.')
)


# __main__.py

# from mock_customers import MOCKCUSTOMERS
# from mock_vendors import MOCKVENDORS

TYPE = "VENDORS"

def main():
    if TYPE == "CUSTOMERS":
        for cust in MOCKCUSTOMERS:
            print("Name : {} , Address : {}".format(cust.name, cust.address))

    elif TYPE == "VENDORS":
        for vend in MOCKVENDORS:
            print("Name : {} , Number : {} Street : {}".format(vend.name, vend.number , vend.street))

    else:
        raise ValueError("Incorrect type {}".format(TYPE))
            
main()

Name : Dough factory , Number : 1 Street : Semolina Court
Name : Farm produce , Number : 14 Street : Country Rd.
Name : Cocoa World , Number : 53 Street : Tropical Blvd.


In [22]:
# Better : USE ADAPTER PATTERN

# classified as structural pattern, ie provides a new way to put the program together.
# convert interface of a class into another one that clients expect
# let's classes work together even though their interfaces aren't compatible
# can provide additional functionality
# Two types of adapters: 
#    -> Object adapters : use composition and can work easily with sub classes as well as parent classes.
#    -> Class adapters : use inheritance and let the adapter over write some of the class' methods
# Generally favour composition over inheritance, because it leads to a flat class structure
# Also known as the wrapper pattern

In [25]:
# Adapter pattern structure
# Screenshot : UML diagram

# Abs Adapter class
# with operation() method
# this abstract base class satisfies the dependency inversion principle which states that you should program 
# towards abstractions rather than implementations(concrete classes) 

# Concrete Adapter class
# the abstract adapter will have one or more concrete implementations.
# all of which are hidden from the client program
# the concrete adapter uses composition to access an object of the adaptee class.

# Adaptee class
# contains AdaptedOperation() method

# AdaptedOperation() method has a signature unknown to the client program, is used by concrete Adapter class
# to present an operation that is known to the client program

# Client program
# client program obtains an instance of the class implementing the abstract adapter (ie concrete adapter)
# and uses its operation() method to do the work.

# what happens behind the scenes in concrete adapter and adaptee class is unkown to the client.

In [7]:
# solution2) object Adapter pattern


# vendor.py
class Vendor:
    
    def __init__(self,name,number,street):
        self._name = name
        self._number = number
        self._street = street 
        
    @property
    def name(self):
        return self._name
    
    @property
    def number(self):
        return self._number
    
    @property
    def street(self):
        return self._street
    
    
# abs_adapter.py
from abc import abstractmethod,ABCMeta,abstractproperty

class AbsAdapter(metaclass=ABCMeta):
    
    def __init__(self,adaptee):
        self._adaptee = adaptee
        
    @property
    def adaptee(self):
        return self._adaptee
    
    @abstractproperty
    def name(self):
        pass
    
    @abstractproperty
    def address(self):
        pass
  

# vend_adapter.py
# from abs_adapter import AbsAdapter

class VendAdapter(AbsAdapter):
    
    @abstractproperty
    def name(self):
        return self.adaptee.name
    
    @abstractproperty
    def address(self):
        return "{} {}".format(self.adaptee.number, self.adaptee.street)
    

    
# mock_vender.py
# from vendor import Vendor
# from vend_adapter import VendAdapter

MOCKVENDORS = (
    VendAdapter(Vendor('Dough factory','1','Semolina Court')),
    VendAdapter(Vendor('Farm produce','14','Country Rd.')),
    VendAdapter(Vendor('Cocoa World','53','Tropical Blvd.'))
)


# __main__.py
# from mock_vendors import MOCKVENDORS

CUSTOMERS = MOCKVENDORS

def main():
    for cust in CUSTOMERS:
        print("Name : {} , Address : {}".format(cust.name, cust.address))


main()


TypeError: Can't instantiate abstract class VendAdapter with abstract methods address, name

In [8]:
# Class adapter structure
# uses multiple inheritance instead of composition

# Screenshot : UML diagram

# Client class
# client program uses original class, with some operation() method defined in original class

# Original class
# contains operation() method

# Adapter class
# subclass original class exposing the same operation() method
# also subclasses Adapted class to use it adaptedoperation() method to prepare the results the client is expecting

# Adapted class
# adaptedoperation() method


# The client program then instantiates the Adapter class instead of Original class.
# and through it transparently interacts with the Adpated class

In [9]:
# solution3

# vendor.py
class Vendor:
    
    def __init__(self,name,number,street):
        self._name = name
        self._number = number
        self._street = street 
        
    @property
    def name(self):
        return self._name
    
    @property
    def number(self):
        return self._number
    
    @property
    def street(self):
        return self._street
    

# customer.py
class Customer:
    
    def __init__(self,name,address):
        self._name = name
        self._address = address
        
    @property
    def name(self):
        return self._name
    
    @property
    def address(self):
        return self._address
    
# vend_adapters.py
# from customer import Customer
# from vendor import Vendor

class VendorAdapter(Vendor,Customer):
    
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
    
    @property
    def address(self): #overwrites address property
        return "{} {}".format(self.adaptee.number, self.adaptee.street)
    

    
# mock_vender.py
# from vend_adapter import VendorAdapter

MOCKVENDORS = (
    VendorAdapter('Dough factory','1','Semolina Court'),
    VendorAdapter('Farm produce','14','Country Rd.'),
    VendorAdapter('Cocoa World','53','Tropical Blvd.')
)


# __main__.py
# from mock_vendors import MOCKVENDORS

CUSTOMERS = MOCKVENDORS

def main():
    for cust in CUSTOMERS:
        print("Name : {} , Address : {}".format(cust.name, cust.address))


main()

AttributeError: 'VendorAdapter' object has no attribute 'adaptee'

In [10]:
# Pros and Cons ; Object Adapter vs Class Adpater

# Object Adapter 
# Composition over inheritance
# More flexible, leads to flatter class hierarchy
# Delegate calls to adaptee
# Works with all adaptee sub classes , provided they can stand in for their parent class.


# Class Adpater
# Inheritance over composition, uses subclassing
# Which allows to specific class or sub class
# over rides adaptee methods
# Dedicated to one adaptee sub class.

In [11]:
# Summary

# Use adapter pattern when you need to use some class but the interface doesn't match to the one you need. 
# ie adapt an interface to the one you need
# Use adapter pattern to create reusable code that works with new, unrelated or unforeseen interfaces.
# Object Adapter: works with several subclasses, useful when it is impractical to sub class each one.
# Class Adapter: works with a specific sub class.
# Object adapter more flexible so prefer using it.

## 3) Decorator Pattern


Adding responsibilities with adapter pattern

In [1]:
# Decorator pattern can help reduce sub classes by adding addition responsibilities at run time instead of compile time.
# eg) for every house class you had a sub class to indicate the color of paint in the kitchen.
# this would lead to explosion of sub classes.
# solution ; decorator pattern

# working example : car dealership
# 3 models
#     -> eco /sport /luxury
# many options
#     -> engine size(4/6 cylinders)
#     -> paint color (white/red/black)

# cost of vehicle depends on the model and options.

In [2]:
# APPROACH 1) Class Based
# abstract car class
# defince concrete class for each model
# subclass each model for option combinations

#  Folder structure
# __main__.py
# CARS
#     -> __init__.py
#     -> abs_car.py
#     -> economy.py
#     -> economy_4cyl_with_vinyl.py
#     -> economy_6cyl_with_vinyl.py

In [5]:
# abs_car.py
from abc import abstractproperty,ABCMeta

class AbsCar(metaclass=ABCMeta):
    
    @abstractproperty
    def description(self):
        pass
    
    @abstractproperty
    def cost(self):
        pass
    
# economy.py   : first concrete model
# from abs_car import AbsCar

class Economy(AbsCar):
    
    @property
    def description(self):
        return "Economy"
    
    @property
    def cost(self):
        return 12000.00
    
    
# economy_4cyl_with_vinyl.py        : subclass with options
# from .economy import Economy

class Economy4CylWithVinyl(Economy):
    
    @property
    def description(self):
        return "Economy ,  white , 4 cylinders ,vinyl upholstery"
    
    @property
    def cost(self):
        return 12000.00
    
    
# economy_6cyl_with_vinyl.py        : subclass with options
# from .economy import Economy

class Economy6CylWithVinyl(Economy):
    
    @property
    def description(self):
        return "Economy ,  white , 6 cylinders ,vinyl upholstery"
    
    @property
    def cost(self):
        return 13500.00
    
    
# __main__.py
# from cars.economy_4cyl_with_vinyl import Economy4CylWithVinyl
# from cars.economy_6cyl_with_vinyl import Economy6CylWithVinyl

def main():
    car1 = Economy4CylWithVinyl()
    car2 = Economy6CylWithVinyl()
    
    print(car1.description, car1.cost)
    print(car2.description, car2.cost)
    
main()

Economy ,  white , 4 cylinders ,vinyl upholstery 12000.0
Economy ,  white , 6 cylinders ,vinyl upholstery 13500.0


In [6]:
# 1 sub-class per options combinations, in this case 36 sub classes
# thousands of combinations in real world therefore subclass explosion and maintenance night mare

In [7]:
# Solution2
# one abstract car class
# concrete class for each model of a car
# This time use properties for the options rather than sub classes

In [9]:
# abs_car.py
from abc import abstractproperty,ABCMeta

class AbsCar(metaclass=ABCMeta):
    
    @abstractproperty
    def description(self):
        pass
    
    @abstractproperty
    def engine(self):
        pass
    
    @abstractproperty
    def paint(self):
        pass
    
    @abstractproperty
    def upholstery(self):
        pass
    
    def cost(self):
        cost = 0.0
        if self.engine == '4 cyl':
            cost += 0.00
        elif self.engine == '6 cyl':
            cost += 1500.00
        if self.paint == 'white':
            cost += 0.00
        elif self.paint == 'black':
            cost += 1000.00
        elif self.paint == 'red':
            cost += 2000.00
        if self.upholstry == 'vinyl':
            cost += 0.00
        elif self.upholstry == 'leather':
            cost += 2000.00

        return cost
    
        
# economy.py   : first concrete model
# from abs_car import AbsCar

class Economy(AbsCar):
    
    def __init__(self,engine,paint,upholstery):
        self._engine = engine
        self._paint = paint
        self._upholstery = upholstery
        
    
    @property
    def description(self):
        return "Economy"
    
    @property
    def engine(self):
        return self._engine

    @property
    def paint(self):
        return self._paint
    
    @property
    def upholstery(self):
        return self._upholstery
    
    @property
    def cost(self):
        return 12000 + super().cost
    
# __main__.py
# from cars.economy import Economy

def main():
    car1 = Economy('4 cyl','black','leather')
    car2 = Economy('6 cyl','red','vinyl')
    
    print(car1.description, car1.cost)
    print(car2.description, car2.cost)
    
main()

TypeError: unsupported operand type(s) for +: 'int' and 'method'

In [10]:
# Here we used properties instead of sub classes to get the desired result.
# pro ; no sub classes and only one concrete sub class per model
# con : more properties to implement and huge if elif chain
# Add new property? make changes to abstract and concrete classes.
# Abstract base class calculates cost therefore violating single resp
# Violates open closed because adding new option would require making changes to abstract and concrete classes both
# Interface segregation violated, cost method would be better with its own interface(ie abstraction)
# Dependancy inversion principle is violated since concrete classes depend on the implementation of cost method
# in abstract base class.
# violation of DRY

In [1]:
# Solution ; Decorator pattern

# Structural pattern, since it offers a new way to put a program together.
# Adds new abilities(properties/options) to an object, dynamically at run time.
# Flexible alternative to sub classing
# Also known as the Wrapper pattern (wrap underlying object through composition). 

In [2]:
# UML diagram : SCREEN-SHOT

# A component is the object we want to decorate.

# Abstract Component class
# has operation() fucntion

# Concrete Component class
# implements abstract component

# Abstract Decorator
# inherits from Abstract component class
# has component attribute

# Concrete Decorator A class
# one or more concrete decorators implement the abstract decorator
# each concrete decorator maintains a reference to the component it decorates, so that it can use that component's operations.
# optionally add newState attribute and NewOperation() methods.

In [4]:
# solution3) use decoration instead of sub classing

# folder structure

# Decorator
#   -> __main__.py
#   -> cars
#         -> __init__.py
#         -> abs_car.py
#         -> economy.py
#         -> luxury.py
#         -> sport.py
#   -> decorators
#         -> __init__.py
#         -> abs_decorator.py
#         -> black.py
#         -> inline4cyl.py
#         -> leather.py
#         -> red.py
#         -> v6.py
#         -> vinyl.py
#         -> white.py

In [8]:
# abs_car.py
from abc import abstractproperty,ABCMeta

class AbsCar(metaclass=ABCMeta):
    
    @abstractproperty
    def description(self):
        pass
    
    @abstractproperty
    def cost(self):
        pass
    

# economy.py   : first concrete model
# from abs_car import AbsCar

class Economy(AbsCar):
    
    @property
    def description(self):
        return "Economy"
    
    @property
    def cost(self):
        return 12000.00
    
    
# luxury.py
# from abs_car import AbsCar

class Luxury(AbsCar):
    
    @property
    def description(self):
        return "Luxury"
    
    @property
    def cost(self):
        return 18000.00
    
    
# sport.py
# from abs_car import AbsCar

class Sport(AbsCar):
    
    @property
    def description(self):
        return "Sport"
    
    @property
    def cost(self):
        return 15000.00
    

# abs_decorator.py
# from cars.abs_car import AbsCar

class AbsDecorator(AbsCar):
    
    def __init__(car):
        self._car = car
        
    @property
    def car(self):
        return self._car
  

# v6.py
# from .abs_decorator import AbsDecorator

class V6(AbsDecorator):
    
    @property
    def description(self):
        return self.car.description + ", V6"
    
    @property
    def cost(self):
        return self.car.cost + 1200.00
    
    
# __main__.py
# from cars.economy import Economy
# from decorators.v6 import V6
# from decorators.black import Black
# from decorators.vinyl import Vinyl

def main():
    car = Economy()
    show(car)
    car = V6(car)
    show(car)
#     car = Black(car)
#     show(car)
    
    
def show(car):
    print("description {}, cost {}".format(car.description,car.cost))
    
main()

description Economy, cost 12000.0


TypeError: __init__() takes 1 positional argument but 2 were given

In [1]:
# Decorator pattern is much more flexible compared to static inheritance.
# run time rather than compile
# Decorator pattern keeps things simple compared to adding a lot of properties.

In [2]:
# Decorator pattern vs Python Decorators

# Decorator pattern
# syntax : class definitions, decorator pattern is implemented with concrete classes that derive from abstract base class. 
# wraps objects
# run time
# add funcitonality to objects


# Python Decorators
# implemented with function definitions and @ syntax
# wraps functions,methods or class definitions.
# compile time
# add functionality to functions,methods or class.

In [5]:
# when to use decorator pattern?

# 1) add new functionality to existing objects.
# better than adding many sub classes with many variations.
# better than adding many properties to a high level class.
# consider using factory or builder pattern to return decorated objects, since you can wind up with many decorators.

## 4) Template method pattern

In [6]:
# deferring algorithm steps with the template method pattern.

# eg) 2 classes with algorithm(steps) to make cake and bread.
# most of the steps are same, though their names and action might differ sometime.
# Template method pattern provides a way to encapsulate the algorithm so that the basic high level steps remain consistent.
# yet allows for different implementation of some steps in a lower,detail sub level.
# in this way pattern encourages code re-use

In [7]:
# Working example

# Imagine you are a passenger bound for New York or Amsterdam
# If you live close to NYC you might take a bus to New York
# If you live in Canada you will fly to New York
# If you live anywhere in North America you'll fly to Amsterdam

# Both bus and plane have a similar purpose
# moving you from point A to point B

# How to model them in python?

In [11]:
# Build two classes
# one for bus trip and another for plane trip



# bus.py
class Bus:
    
    def __init__(self,destination):
        self._destination = destination 
    
    def bus_trip(self):
        self.start_diesel()
        self.leave_terminal()
        self.drive_to_destination()
        self.arrive_at_destination()
    
    def start_diesel(self):
        print("starting the diesel engine")
    
    def leave_terminal(self):
        print("Leaving terminal")
    
    def drive_to_destination(self):
        print("Driving")
    
    def arrive_at_destination(self):
        print("Arriving at {}".format(self._destination))

# airplane.py
class Airplane:
    
    def __init__(self,destination):
        self._destination = destination 
    
    def plane_trip(self):
        self.start_gas_turbine()
        self.leave_terminal()
        self.fly_to_destination()
        self.land_at_destination()
    
    def start_gas_turbine(self):
        print("starting the gas turbine engine")
    
    def leave_terminal(self):
        print("Taking off")
    
    def fly_to_destination(self):
        print("Flying")
    
    def land_at_destination(self):
        print("Landing at {}".format(self._destination))
        
        
# __main__.py
# from bus import Bus
# from airplane import Airplane

def main():
    take_bus('New York')
    take_plane('Amsterdam')
    
def take_bus(destination):
    print("taking bus to {}".format(destination))
    bus = Bus(destination)
    bus.bus_trip()

def take_plane(destination):
    print("flying to {}".format(destination))
    plane = Airplane(destination)
    plane.plane_trip()


main()

taking bus to New York
starting the diesel engine
Leaving terminal
Driving
Arriving at New York
flying to Amsterdam
starting the gas turbine engine
Taking off
Flying
Landing at Amsterdam


In [12]:
# violation of DRY
# solution Template method pattern

In [1]:
## Template method
# behavioral method, so it changes how classes interface with each other
# defines algorithm skeleton, deffering some of the steps to sub classes.
# the overall structure of the algorithm doesn't change however.
# starts with an abstract base class with 3 types of methods
# a) abstract methods and these must be implemented by all the sub classes
# b) concrete methods : sub classes(derived classes) can use deafult implementation or over write them
# c) hooks : empty, do nothing but can be over written by derived classes if they need to do something specific.
#  Order of the methods is fixed
# Fixed order is encapsulated in a separate template method

# SCREENSHOT ; UML DIAGRAM
# Template method structure

# AbstractClass ; abstract base class with 3 methods
# primitive_operation1() method
# primitive_operation2() method
# template_method() : determines the order of execution of the primitive operations
# the primitive operation methods in the abstract base class can be abstract,concrete or hooks.

# Class1 and Class2
# are concrete classes which derive from the abstract base class
# They must implement the abstract methods
# They may over ride the concrete and hook methods
# the the template_method() present in the abstract class defines the order of calling methods which can't be changed here.

In [3]:
# working example

# abs_transport.py
from abc import ABCMeta, abstractmethod

class AbsTransport(metaclass=ABCMeta):
    
    def __init__(self,destination):
        self._destination = destination 
    
    # take_trip is the template method which defines the order of execution
    def take_trip(self):
        self.start_engine()
        self.leave_terminal()
        self.entertainment()
        self.travel_to_destination()
        self.arrive_at_destination()
    
    @abstractmethod
    def start_engine(self): # abstract therefore must be implemented in any sub class
        pass
    
    def leave_terminal(self): # concrete method can be over written
        print("Leaving terminal")
    
    @abstractmethod
    def travel_to_destination(self): 
        print("Travelling")
    
    def entertainment(self): # concrete but no implementation, therefore hook
        pass
    
    def arrive_at_destination(self):
        print("Arriving at {}".format(self._destination))

        
        
# airplane.py
# from abs_transport import AbsTransport

class Airplane(AbsTransport):

    # necessary implementation of abstract method
    def start_engine(self):
        print("starting the gas turbine engine")
    
    # over writes concrete method 
    def leave_terminal(self):
        print("Taking off")
    
    # necessary implementation of abstract method
    def travel_to_destination(self):
        print("Flying")
    
    # over writes hook method
    def entertainment(self):
        print("Playing in flight movie")
    
    # over writes concrete method
    def land_at_destination(self):
        print("Landing at {}".format(self._destination))

        
# bus.py
# from abs_transport import AbsTransport

class Bus(AbsTransport):
    
    def start_engine(self):
        print("starting the diesel engine")
        
    def travel_to_destination(self):
        print("Driving")

        
        
# __main__.py
# from bus import Bus
# from airplane import Airplane

def main():
    travel('New York',Bus)
    travel('Amsterdam',Airplane)
    
def travel(destination,transport):
    print("\n")
    print("Taking {} to {}".format(transport.__name__,destination))
    means = transport(destination)
    means.take_trip()


main()



Taking Bus to New York
starting the diesel engine
Leaving terminal
Driving
Arriving at New York


Taking Airplane to Amsterdam
starting the gas turbine engine
Taking off
Playing in flight movie
Flying
Arriving at Amsterdam


In [4]:
## pros of template method

# code reuse
# ensures required steps are implemented
# allows for over riding some steps
# hooks allow to inject special requirements at pre defined points.
# enforces the algorithm order and structure
# not useful if the algorithm itself must vary

# when to use?
# recognize similarities in classes that implement equivalent algorithms
# take advantage of the 3 types of methods in the pattern
# abstract ; must be implemented by all sub classes
# concrete ; but overridable
# hook ; for additional functionality 

## 5) Iterator pattern

In [5]:
# collections example
# list of ingredients for menu
# db for order processing
# employee table of a company

# retrieve items from collections(usually one at a time)
# that operation is called iteration
# iterating over collections mean returning objects to the caller one by one
# HIDE THE IMPLEMENTATION

# working exmaple: employee collection
# collection holds zero or more employee objects
# clients iterate over that collection
# in turn collection need to expose a method for performing that iteration
# while hiding the implemtation of the collection

In [6]:
# solution1
# collection of employees could be list,set,dict,tree etc..
# this information is hidden from the client
#  hiding implementation is a good principle of OOP


# employee.py
class Employee:
    
    def __init__(self,empid,name,hiredate):
        self.empid = empid,
        self.name = name,
        self.hiredate = hiredate

    
# employee_collection.py
class Employees:
    _employees = {}
    _headcount = 0
    
    def add_employee(self,employee):
        self._headcount += 1
        self._employees[self._headcount] = employee
    
    def get_employee(self,i):
        return self._employees[i]
    
    @property
    def headcount(self):
        return self._headcount
    

# testdata.py
# from employee import Employee
# from employee_collection import Employees
# from department import Department
# from department_collection import Departments
from datetime import datetime


TESTEMPLOYEES = (
    (1,'Douglas Adam',datetime(1947,7,6)),
    (2,'Sherlock Holmes',datetime(1887,3,16)),
    (3,'Albert Einstein',datetime(1857,11,25)),
    (4,'Sir John',datetime(1915,8,1)),
    (5,'T Roosevelt',datetime(1901,9,14))
)

employees = Employees()
for empid,name,hiredate in TESTEMPLOYEES:
    employees.add_employee(Employee(empid,name,hiredate))

# __main__.py
# from testdata import employees

def main():
    print("employees")
    
    for i in range(1,employees.headcount+1):
        employee = employees.get_employee(i) 
        print("Employee id {}, Name {} , Date of hire {}".format(employee.empid,employee.name,employee.hiredate))

        
main()

employees
Employee id (1,), Name ('Douglas Adam',) , Date of hire 1947-07-06 00:00:00
Employee id (2,), Name ('Sherlock Holmes',) , Date of hire 1887-03-16 00:00:00
Employee id (3,), Name ('Albert Einstein',) , Date of hire 1857-11-25 00:00:00
Employee id (4,), Name ('Sir John',) , Date of hire 1915-08-01 00:00:00
Employee id (5,), Name ('T Roosevelt',) , Date of hire 1901-09-14 00:00:00


In [7]:
# Since there is no other method exposed we are using the get_employee method to iterate over the collection.
#  add departments too

# solution1 + departments
# collection of employees could be list,set,dict,tree etc..
# this information is hidden from the client
#  hiding implementation is a good principle of OOP


# employee.py
class Employee:
    
    def __init__(self,empid,name,hiredate):
        self.empid = empid,
        self.name = name,
        self.hiredate = hiredate

        
# department.py
class Department:
    
    def __init__(self,deptid,name,date_established):
        self.deptid = deptid,
        self.name = name,
        self.date_established = date_established
        
    
# employee_collection.py
class Employees:
    _employees = {}
    _headcount = 0
    
    def add_employee(self,employee):
        self._headcount += 1
        self._employees[self._headcount] = employee
    
    def get_employee(self,i):
        return self._employees[i]
    
    @property
    def headcount(self):
        return self._headcount
    

# department_collection.py
class Departments:
    _departments = [] # list instead of dict used in employees
    
    def add_department(self,department):
        self._departments.append(department)
    
    def get_department(self,i):
        return self._departments[i]
    
    @property
    def departments_range(self):
        return (o,len(self._departments) -1)
    

# testdata.py
# from employee import Employee
# from employee_collection import Employees
# from department import Department
# from department_collection import Departments
from datetime import datetime


TESTEMPLOYEES = (
    (1,'Douglas Adam',datetime(1947,7,6)),
    (2,'Sherlock Holmes',datetime(1887,3,16)),
    (3,'Albert Einstein',datetime(1857,11,25)),
    (4,'Sir John',datetime(1915,8,1)),
    (5,'T Roosevelt',datetime(1901,9,14))
)

employees = Employees()
for empid,name,hiredate in TESTEMPLOYEES:
    employees.add_employee(Employee(empid,name,hiredate))

# __main__.py
# from testdata import employees

def main():
    print("employees")
    
    for i in range(1,employees.headcount+1):
        employee = employees.get_employee(i) 
        print("Employee id {}, Name {} , Date of hire {}".format(employee.empid,employee.name,employee.hiredate))

    
    print("Departments")
    
    for i in range(*departments.departments_range):
        dept = departments.get_departments(i)
        print("Department id {}, Name {} , Date established {}".format(department.deptid,dept.name,dept.date_established))
        
    
    print_summary(employees)
    print_summary(departments)
    
    def print_summary(collection):
#         need to know the type of collection
#         without a runtime test for type you can't interate 
        pass
    
main()

employees
Employee id (1,), Name ('Douglas Adam',) , Date of hire 1947-07-06 00:00:00
Employee id (2,), Name ('Sherlock Holmes',) , Date of hire 1887-03-16 00:00:00
Employee id (3,), Name ('Albert Einstein',) , Date of hire 1857-11-25 00:00:00
Employee id (4,), Name ('Sir John',) , Date of hire 1915-08-01 00:00:00
Employee id (5,), Name ('T Roosevelt',) , Date of hire 1901-09-14 00:00:00
Departments


NameError: name 'departments' is not defined

In [8]:
#solution ITERATOR PATTERN

# behavioral pattern
# in this case add new abilities to the collection
# iterate over the elements of collection without exposing the underlying representation
# this preserves encapsulation
# also known as the cursor pattern

# Iterator pattern structure
# SCREENSHOT : UML DIAGRAM

# Iterable ; abstract base class
# create_iterator()  method : in the above example employees are the iterable

# Collection : concrete class
# provides implementation of create_iterator() method

# Iterator : abstract base class
# standard method to navigate the collection
# currenti_item(), is_done(), first(), next()

# ConcreteIterator ; implementation of abstract "Iterator" class

# here the employees and departments have two different ways of iteration over their collection

# Client ; class
# uses the concrete methods which have been implemented to iterate over the collection.
# create_iterator() , next() , is_done() , current_item() , first() methods

In [9]:
# python iterators

# sequence iterator : __getitem__()
# callable object : __iter__() and next()

# iterable,iterator and sequence abstract base classes available in collections module.

In [5]:
# build iterators for both employees and departments collection

# Since there is no other method exposed we are using the get_employee method to iterate over the collection.
#  add departments too

# solution1 + departments
# collection of employees could be list,set,dict,tree etc..
# this information is hidden from the client
#  hiding implementation is a good principle of OOP


# employee.py
class Employee:
    
    def __init__(self,number,name,date):
        self.number = number,
        self.name = name,
        self.date = date

        
# department.py
class Department:
    
    def __init__(self,number,name,date):
        self.number = number,
        self.name = name,
        self.date = date
        
    
# employee_collection.py
from collections import Iterator

class Employees(Iterator):
    _employees = {}
    _headcount = 0
    _empid = 0
    
    def add_employee(self,employee):
        self._headcount += 1
        self._employees[self._headcount] = employee
    
    def __iter__(self):
        self._empid = 0
        return self
    
    def __next__(self):
        if self._empid < self._headcount:
            self._empid += 1
            return self._employees[self._empid]
        else:
            return StopIteration

# department_collection.py
from collections import Sequence


class Departments(Sequence):
    _departments = [] # list instead of dict used in employees
    
    def add_department(self,department):
        self._departments.append(department)
    
    def __getitem__(self,item):
        return self._departments[item]
    
    def __len__(self):
        return len(self._departments)
    

# testdata.py
# from employee import Employee
# from employee_collection import Employees
# from department import Department
# from department_collection import Departments
from datetime import datetime


TESTEMPLOYEES = (
    (1,'Douglas Adam',datetime(1947,7,6)),
    (2,'Sherlock Holmes',datetime(1887,3,16)),
    (3,'Albert Einstein',datetime(1857,11,25)),
    (4,'Sir John',datetime(1915,8,1)),
    (5,'T Roosevelt',datetime(1901,9,14))
)

employees = Employees()
for number,name,date in TESTEMPLOYEES:
    employees.add_employee(Employee(number,name,date))

    
# TESTDEPARTMENTS = (
#     (1,'Douglas Adam',datetime(1947,7,6)),
#     (2,'Sherlock Holmes',datetime(1887,3,16)),
#     (3,'Albert Einstein',datetime(1857,11,25)),
#     (4,'Sir John',datetime(1915,8,1)),
#     (5,'T Roosevelt',datetime(1901,9,14))
# )

# departments = Departments()
# for number,name,date in TESTDEPARTMENTS:
#     departments.add_department(Department(number,name,date))
    
    
# __main__.py
# from testdata import employees,departments

def main():
    print("employees")
    print_summary(employees)

    print("Departments")
    print_summary(departments)
    
def print_summary(collection):
    for item in collection:
        print("Item id {}, Name {}, Date{}".format(item.number,item.name,item.date))
    
main()

employees
Item id (1,), Name ('Douglas Adam',), Date1947-07-06 00:00:00
Item id (2,), Name ('Sherlock Holmes',), Date1887-03-16 00:00:00
Item id (3,), Name ('Albert Einstein',), Date1857-11-25 00:00:00
Item id (4,), Name ('Sir John',), Date1915-08-01 00:00:00
Item id (5,), Name ('T Roosevelt',), Date1901-09-14 00:00:00


AttributeError: type object 'StopIteration' has no attribute 'number'

In [6]:
# what if we have two simultaneous iterators over the same collection


# bug.py
# from testdata import employees

i1 = iter(employees)
i2 = iter(employees)

assert i1 == i2

for _ in range(5):
    print("{} {}".format(next(i1).number,next(i2).number))

(1,) (2,)
(3,) (4,)


AttributeError: type object 'StopIteration' has no attribute 'number'

In [8]:
# multiple iteration support


# employee_collection.py
from collections import Iterator,Iterable

class Employees(Iterator):
    _employees = {}
    _headcount = 0
    _empid = 0
    
    def add_employee(self,employee):
        self._headcount += 1
        self._employees[self._headcount] = employee
    
    def __iter__(self):
        return EmployeesIterator(self._employees,self._headcount)
        
    
class EmployeesIterator(Iterator):
    
    def __init__(self,employees,headcount):
        self._employees = employees
        self._headcount = headcount
        self._empid = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._empid < self._headcount:
            self._empid += 1
            return self._employees[self._empid]
        else:
            return StopIteration
        
        
# bug.py
# from testdata import employees

i1 = iter(employees)
i2 = iter(employees)

# assert i1 == i2

for _ in range(5):
    print("{} {}".format(next(i1).number,next(i2).number))

(1,) (2,)
(3,) (4,)


AttributeError: type object 'StopIteration' has no attribute 'number'

In [9]:
# skipped

In [11]:
# another solution : USE GENERATOR

# multiple iteration support


# employee.py
class Employee:
    
    def __init__(self,number,name,date):
        self.number = number,
        self.name = name,
        self.date = date

        
# department.py
class Department:
    
    def __init__(self,number,name,date):
        self.number = number,
        self.name = name,
        self.date = date
        

# employee_collection.py
from collections import Iterator

class Employees(Iterator):
    _employees = {}
    _headcount = 0
    _empid = 0
    
    def add_employee(self,employee):
        self._headcount += 1
        self._employees[self._headcount] = employee
    
    def __iter__(self):
        return (e for e in self._employees.values())
        

# department_collection.py
from collections import Sequence


class Departments(Sequence):
    _departments = [] # list instead of dict used in employees
    
    def add_department(self,department):
        self._departments.append(department)
    
    def __iter__(self):
        return (d for d in self._departments)

    

# testdata.py
# from employee import Employee
# from employee_collection import Employees
# from department import Department
# from department_collection import Departments
from datetime import datetime


TESTEMPLOYEES = (
    (1,'Douglas Adam',datetime(1947,7,6)),
    (2,'Sherlock Holmes',datetime(1887,3,16)),
    (3,'Albert Einstein',datetime(1857,11,25)),
    (4,'Sir John',datetime(1915,8,1)),
    (5,'T Roosevelt',datetime(1901,9,14))
)

employees = Employees()
for number,name,date in TESTEMPLOYEES:
    employees.add_employee(Employee(number,name,date))

    
# TESTDEPARTMENTS = (
#     (1,'Douglas Adam',datetime(1947,7,6)),
#     (2,'Sherlock Holmes',datetime(1887,3,16)),
#     (3,'Albert Einstein',datetime(1857,11,25)),
#     (4,'Sir John',datetime(1915,8,1)),
#     (5,'T Roosevelt',datetime(1901,9,14))
# )

# departments = Departments()
# for number,name,date in TESTDEPARTMENTS:
#     departments.add_department(Department(number,name,date))
    
    
# __main__.py
# from testdata import employees,departments

def main():
    print("employees")
    print_summary(employees)

    print("Departments")
    print_summary(departments)
    
def print_summary(collection):
    for item in collection:
        print("Item id {}, Name {}, Date{}".format(item.number,item.name,item.date))
    
main()
        
# multiple.py
# from testdata import employees

i1 = iter(employees)
i2 = iter(employees)

# assert i1 == i2

for _ in range(5):
    print("{} {}".format(next(i1).number,next(i2).number))

TypeError: Can't instantiate abstract class Employees with abstract methods __next__

In [12]:
#  3 consequences

# 1) simple,standard interface to process members of a collection
# then any program can use for loop or comprehension to process the items in that collection
# 2) Collection can implement the iterator in any way it needs to
# the implementations can vary widely, even for the same collection
# eg) n way tree can be process breadth first or depth first
# 3) Multiple active , independent iterators

# When to use iterator pattern?
# Iterator over a collection, and preserve the encapsulation and not exposing it's internal implementation
# use it if you want to support multiple active iterators, using a generator can help you do this easily
# use iterator if you want to provide a uniform interface.

## 6) Composite pattern

In [1]:
# This pattern is all about trees
# hierarachy, family trees, nested groups, evolution taxanomy are all examples of tree structures
# part whole hierarchies : a part resembles the whole
# composite pattern handles such hierarchies in code
# uniform code for the part or whole.

In [5]:
# working example
# A family tree or a set of family trees
# which consists of parents and children
# find out the oldest person

# person.py
class Person:
    name = None
    birthdate = None
    
    def __init__(self,name,birthdate):
        self.name = name
        self.birthdate = birthdate

        
# family.py
from collections import Iterable

class Family(Iterable):
    members = []
    
    def __init__(self,members):
        self.members = members

    def __iter__(self):
        return iter(self.members)
    
    
# __main__.py
from datetime import date
# from person import Person
# from family import Family

def main():
    family = Family([
        Person("bob",date(1970,3,14)),
        Person("bar",date(1991,7,1)),
        Person("foo",date(1995,2,4)),
        Person("ford",date(1978,6,24)),
        Person("arthur",date(1999,12,6))
    ])
    
    singles = [
        Person("marvin",date(1991,1,1)),
        Person("douglas",date(1993,1,3))
    ]
    
    oldest = None
    earliest_date = date.max
    for m in family:
        if m.birthdate < earliest_date:
            oldest = m
            earliest_date = m.birthdate
            
    for s in singles:
        if s.birthdate < earliest_date:
            oldest = s
            earliest_date = s.birthdate
            
    max_age = (date.today() - earliest_date).days / 365.2425
    print("oldest person {}, Age {:6.2f}".format(oldest.name,max_age))
    
    
main()

oldest person bob, Age  48.98


In [6]:
# two loops for members and singles
# if you want to add another generation ie another level to the tree
# you'll need to change it to recursive function for each level

# better use Composite pattern

In [8]:
# Composite pattern is a structural pattern
# a way of putting objects together in tree structures
# these structures represent part whole hierarchies
# in the first demo person,family,singles etc are the parts put together they form a family tree
# organize these objects into a tree structure, that would mean 
# clients can handle individual objects like person object
# and collections of objects like the family object or list of singles
# using the same code without changing the tree depth

# UML DIAGRAM

<img src="uml/composite_pattern.png">

In [9]:
# AbsComponent ; abstract base class
# Clients use this abstract component interface to handle the objects in a tree
# the Abscomponent must be implemented by both the Leaf and Composite concrete classes
# contains add_component(), get_child(), operation(), remove_component() methods

# Instances(objects) of Lead and Composite classes are called nodes

# Leaf class
# a leaf node is at the bottom of the tree
# they have no memebers yet they inherit the get_child, add_component and remove_component methods
# which is why the abstract component implemenets some default operation for these methods.
# cotains only one method operation() will just return some information about that node

# Composite class
# a composite node is a sub tree which will have a set of children, some of which maybe empty
# and those children can be leaves of other composites
# so the operation method of composite class is an iterator and follows the iterator pattern

In [15]:
# working example :
# create a tree holding families and singles
# Family and Person class will now have AbsComposite interface


# abs_composite.py
from abc import ABCMeta,abstractmethod

class AbsComposite(metaclass=ABCMeta):
        
    @abstractmethod
    def get_oldest(self):
        pass

    
# person.py
class Person(AbsComposite):
    
    def __init__(self,name,birthdate):
        self.name = name
        self.birthdate = birthdate

    def get_oldest(self):
        return self
    

# replaces the family class that we had before
# corresponds to the Composite concrete class

# tree.py 
from collections import Iterable
from functools import reduce
from datetime import date
# from abs_composite import AbsComposite

class Tree(Iterable,AbsComposite):
    
    def __init__(self,members):
        self.members = members
        
    def __iter__(self):
        return iter(self.members)
    
    # takes pair of elements of the tree and returns the greater, this is recursive depth first traversal of tree
    def get_oldest(self):
        def f(t1,t2):
            t1_,t2_ = t1.get_oldest() ,t2.get_oldest()
            return t1_ if t1.birthdate < t2.birthdate else t2_
           
        return reduce(f,self,NullPerson())
    

class NullPerson(AbsComposite):
    name = None
    birthdate = date.max
    
    def get_oldest(self):
        return self

    
# __main__.py
from datetime import date
# from person import Person
# from tree import Tree

def main():
    hitchhikers = Tree([
        Person("bob",date(1970,3,14)),
        Person("bar",date(1991,7,1)),
        Person("foo",date(1995,2,4)),
        Person("ford",date(1978,6,24)),
        Person("arthur",date(1965,12,6))
    ])
    
    singles = Tree([
        Person("marvin",date(1991,1,1)),
        Person("douglas",date(1993,1,3))
    ])
    
    loner = Person("Dirk",date(1990,6,7))
    
    
    tree1 = Tree([hitchhikers])
    tree2 = Tree([singles,loner])
    tree3 = Tree([tree1,tree2])
    
    
    for tree in tree1,tree2,tree3:
        oldest = tree.get_oldest()
        max_age = (date.today() - oldest.birthday).days / 365.2425
        print("oldest person {}, Age {:6.2f}".format(oldest.name,max_age))

    
main()

AttributeError: 'Tree' object has no attribute 'birthdate'

In [1]:
## Consequences

# Organises data into tree structure
# this permits unified access to subtrees and leaf nodes
# which makes for simplified client code
# No need to do run time checking for different components
# Easy to add new components that implement the same interface
# without changing the client code.
# Follows open closed but violates single resp principle

In [2]:
### Summary

# When to use composite pattern?
# when you data fits in a tree like structure
# and client code can treat leafs and sub trees uniformly
# enhance composite pattern so that child nodes maintain parent references.
# It's also possible to share components, which can reduce storage requirements
# can make your design too general, since it violates the single resp principle

## 7) State Pattern

In [3]:
# Altering behaviour with the state pattern
# uses STATE DIAGRAMS(used to model real and theoretical problems)
# eg) kitchen could be in tidy state, messy state, active state, clean up state

In [5]:
# Working example : Shopping Cart
# Cart could be in a physical super market or e commerce marketplace
# Cart could be in various states:
#      -> Empty
#      -> Containing some items
#      -> At the checkout
#      -> Paid for

# Transitions: getting from one state to another
#      -> Adding or removing items
#      -> Checking out
#      -> Paying for your purchases

# but not all transitions are possible in each state

Shopping cart state diagram:

Empty -> Non-Empty -> Checkout -> Paid for

<img src="uml/shopping_cart_state_diagram.png">

In [6]:
# In the above example
# States are empty, non-empty, check out, paid for
# Transitions are Add item, Remove item, Add or Remove item, Go to Check out, Empty the cart, Pay for it

In [7]:
## demo : model the shopping cart

# use one variable to track the state
# create methods for state transitions
# run the model to test out your solution

In [10]:
# shopping_cart.py
EMPTY = 0
NON_EMPTY = 1
AT_CHECKOUT = 2
PAID_FOR = 3

class ShoppingCart:
    
    def __init__(self):
        self.state = EMPTY
        self._items = 0
        
    def add_item(self):
        if self.state == EMPTY:
            print("you have added the first item")
            self.state = NON_EMPTY
            self._items += 1
            
        elif self.state == NON_EMPTY:
            self._items += 1
            print("you now have {} items in your cart".format(self._items))
        
        elif self.state == AT_CHECKOUT:
            print("You can't add new items at checkout")
            
        else: # state = PAID_FOR
            print("You can't add items after payment")
    
    
    def remove_item(self):
        if self.state == EMPTY:
            print("your cart is empty, nothing to remove")
    
        elif self.state == NON_EMPTY:
            self._items -= 1
            print("you now have {} items in your cart".format(self._items))
            if self._items == 0:
                self.state = EMPTY
        
        elif self.state == AT_CHECKOUT:
            self.state -= 1
            print("you now have {} items in your cart".format(self._items))
            if self._items == 0:
                self.state = EMPTY
            else:
                self.state = NON_EMPTY
            
        else: # state = PAID_FOR
            print("You can't remove items after payment")
            
        
    def checkout(self):
        if self.state == EMPTY:
            print("your cart is empty, go back and shop")
    
        elif self.state == NON_EMPTY:
            print("you now have {} items in your cart".format(self._items))
            self.state = AT_CHECKOUT
        
        elif self.state == AT_CHECKOUT:
            print("already at checkout")
            
        else: # state = PAID_FOR
            print("You can't go back to checkout after payment")
            
        
    def paid_for(self):
        if self.state == EMPTY:
            print("your cart is empty, how did you get here?")
    
        elif self.state == NON_EMPTY:
            print("you must go to checkout for payment")

        elif self.state == AT_CHECKOUT:
            print("you have paid for {} items".format(self._items))
            self._items = 0
            self.state = EMPTY
                        
        else: # state = PAID_FOR
            print("You have already paid for your purchases")
            
            
# __main__.py
# from shopping_cart import ShoppingCart

def main():
    cart = ShoppingCart()
    cart.add_item()
    cart.remove_item()
    cart.add_item()
    cart.add_item()
    cart.add_item()
    cart.remove_item()
    cart.checkout()
    cart.paid_for()
    cart.add_item()
    cart.checkout()
    cart.paid_for()


main()

you have added the first item
you now have 0 items in your cart
you have added the first item
you now have 2 items in your cart
you now have 3 items in your cart
you now have 2 items in your cart
you now have 2 items in your cart
you have paid for 2 items
you have added the first item
you now have 1 items in your cart
you have paid for 1 items


In [11]:
# for each transition, you have to check each of the four states
# if you had to add a 5th state called save, so shoppers can pause their shopping and resume later
# to do that you would have to modify each transition method to add if check for each state and add appropriate action.
# violates the open closed principle
# too many if else
# solution : STATE PATTERN

In [13]:
# State pattern is a behavioral pattern, it controls how your program operates
# pattern operates in some context
# instead of putting transition methods in the context object, we put them in the state classes using one class for each state
# context delegates transitions to the state objects
# while the clients only interface with the context
# State pattern's structure

<img src="uml/state_pattern.png">

In [14]:
## Context class
# client interface ie the shopping cart in the demo
# attribtues : state , keeps a reference to the current state, and that reference is used for every request(method which are the state transitions) : state->Handle()
# methods : request() = state transitions

# AbsState class
# all states derive from this abstract base class
# which defince abstract method handle() , for each type of request

# ConcreteState class 
# each concrete state class implements this handle() method, to handle each type of request

In [3]:
## Implementation of the state pattern

# create a shopping cart context object
# create state classes
# these state classes and their objects need to handle transition requests 


# abs_state.py
from abc import ABCMeta,abstractmethod


class AbsState(metaclass = ABCMeta):
    
    def __init__(self,context):
        self._cart = context
        
    # 5 abstract methods one for each of the transitions
    @abstractmethod
    def add_item(self):
        pass
    
    @abstractmethod
    def remove_item(self):
        pass
    
    @abstractmethod
    def checkout(self):
        pass
    
    @abstractmethod
    def pay(self):
        pass
    
    def empty_cart(self):
        pass

      
# ONE CLASS FOR EACH OF THE 4 STATES A SHOPPIG CART CAN BE IN ; empty/non-empty/check-out/paid-for


# empty.py
# from abs_state import AbsState

class Empty(AbsState):
    
    def add_item(self):
        self._cart.items += 1
        print("you have added first item")
        self._cart.state  = self._cart.not_empty
    
    def remove_item(self):
        print("your carty is empty nothing to remove")
        
    def checkout(self):
        print("your cart is empty. .go shopping")
    
    def pay(self):
        print("your cart is empty , how did you get here?")
    
    def empty_cart(self):
        print("your cart is already empty")


# non_empty.py
# from abs_state import AbsState

class NotEmpty(AbsState):
    
    def add_item(self):
        self._cart.items += 1
        print("you have {} items in your shopping cart".format(self._cart.items))
        
    def remove_item(self):
        self._cart.items -= 1
        if self._cart.items:
            print("you have {} items in your shopping cart".format(self._cart.items))
        else:
            print("your cart is empty, try again")
            self._cart.state = self._cart.empty
    
    def checkout(self):
        print("Done shopping. Let's check out")
        self._cart.state = self._cart.check_out
    
    def pay(self):
        print("you have to go to checkout to pay")
    
    def empty_cart(self):
        print("you have already emptied the cart at the checkout")


# check_out.py
# from abs_state import AbsState

class AtCheckOut(AbsState):
    
    def add_item(self):
        print("you can't add items at checkout")
    
    def remove_item(self):
        self._cart.items -= 1
        if self._cart.items:
            print("you now have {} items in your cart".format(self._cart.items))
        else:
            print("your cart is empty again")
            self._cart.state = self._cart.empty
    
    def checkout(self):
        print("you're already at the checkout")
    
    def pay(self):
        print("you have paid for {} items".format(self._cart.items))
        self._cart.state = self._cart.paid_for
    
    def empty_cart(self):
        self._cart.items = 0
        self._cart.state = self._cart.empty

    
# paid_for.py
# from abs_state import AbsState

class PaidFor(AbsState):
    
    def add_item(self):
        print("you've already paid for your purchases. want to shop more? take a new cart")
    
    def remove_item(self):
        print("you've already paid for your purchases, can't remove any items from your cart")
    
    def checkout(self):
        print("why are you back here? you've already paid for your purchases")
    
    def pay(self):
        print("you've already paid for your purchases. Can't pay twice")
    
    def empty_cart(self):
        print("you've already paid for your purchases. time to go home")

        
# shopping_cart.py
# from empty import Empty
# from non_empty import NonEmpty
# from check_out import AtCheckOut
# from paid_for import PaidFor
  
class ShoppingCart:
    
    def __init__(self):
        self.empty = Empty(self)
        self.not_empty = NotEmpty(self)
        self.check_out = AtCheckOut(self)
        self.paid_for = PaidFor(self)
        
        self.items = 0
        self.state = self.empty
        
    def add_item(self):
        self.state.add_item()
        
    def remove_item(self):
        self.state.remove_item()
        
    def checkout(self):
        self.state.checkout()
        
    def pay(self):
        self.state.pay()
     
    def empty_cart(self):
        self.state.empty_cart()

                
# __main__.py
# from shopping_cart import ShoppingCart

def main():
    print("==> First cart")
    cart = ShoppingCart()
    cart.add_item()
    cart.remove_item()
    cart.add_item()
    cart.add_item()
    cart.add_item()
    cart.remove_item()
    cart.checkout()
    cart.pay()
    
    # go shopping again
    print("==> Second cart")
    cart = ShoppingCart()
    cart.add_item()
    cart.add_item()
    cart.checkout()
    cart.empty_cart()
    cart.add_item()
    cart.checkout()
    cart.pay()

    print("==> error for adding item after paying")
    cart.add_item()
    
    
    
main()

==> First cart
you have added first item
your cart is empty, try again
you have added first item
you have 2 items in your shopping cart
you have 3 items in your shopping cart
you have 2 items in your shopping cart
Done shopping. Let's check out
you have paid for 2 items
==> Second cart
you have added first item
you have 2 items in your shopping cart
Done shopping. Let's check out
you have added first item
Done shopping. Let's check out
you have paid for 1 items
==> error for adding item after paying
you've already paid for your purchases. want to shop more? take a new cart


In [4]:
## Consequences

# Encapsulates state specific behaviour.
# eg) behaviour of the shopping cart in the empty state was different that it's behaviour in the non-empty state
# yet the shopping cart was unaware of the current state or when the state changed(ie transition)
# this makes it easy to add new states
# you might be asked to add save or suspended state to the cart
# Possible to add default behaviour to the abstract base class
# so that you only need to implement methods(state transition methods) in the sub classes (Concrete State sub-classes)
# Distributes behaviour across state sub classes
# Makes state transitions explicit, without complicating code in the context ie the Shopping cart
# State objects can be shared if there are no instance variables, saving memory
# State pattern doesn't dictate where transitions are defined, therefore flexible transition definitions
# we have defined them in sub classes but you can also define them in the context ie shopping cart
# especially if the transitions are fixed
# States can be created at transition time as well

In [5]:
## Summary
# when to use the state pattern?
# when an object's behavior depends upon it's state, and that behavior changes as the state changes
# use to replace if/elif/else statements, when you are checking an object's state

## 8) Proxy Pattern

In [1]:
## Controlling access with the proxy pattern

# proxy stands between the client code and the actual object you need to access.
# FOUR types of implementation of the proxy pattern
# 1) Remote proxy
# provides local rep for an object in a different address space.
# eg) web client that uses a db on a separate machine, this allows the db to kept behind the corporate fire wall
# while the webserver is in the dmz base in the internet, this enhances security by enhancing and controlling access to db.
# 2) Virtual proxy
# creates objects on demand.
# eg) your client needs to give information from db on another server, that query may run long or return 10000s of rows to fully populate the object
# a virtual proxy can provide a way to return the data in a lazy fashion, only when needed
# 3) Protection proxy
# controls access to the original object
# this is useful when access rights of clients depend upon user id or time etc.
# eg) db restricted access to admin, employee etc..
# 4) Smart Reference proxy
# perform additional actions when an object is accessed
# eg) reference count, so that an object can be released when it is no longer accessed

# DBMSs uses all these types of proxies
# remote proxies to access db on another server
# virtual proxies are used with buffering to minimize IO
# protection proxies are used to limit access to objects and parts of objects
# eg)limit access to certain rows and columns in a db table
# smart ref proxies are used for logging, locking, maintaing stats etc..

In [2]:
## working example

# Employee object, you need to control access to it
# sensitive info : birthdate and salary

# Access control object
# employee id and flag to check if the accessor can see personal information or not

# Client program
# to access employee objects, while restricting access to private information
# according to the entries in the access control object

In [6]:
# employee.py

class Employee:
    
    def __init__(self,empid,name,birthday,salary):
        self.empid = empid
        self.name = name
        self.birthday = birthday
        self.salary = salary # restrict access to birthdate and salary of the employee object


# access_control.py

class AccessControl:
    
    def __init__(self,empid,can_see_personal):
        self.empid = empid
        self.can_see_personal = can_see_personal # Bool if true allow request from that empid to see personal info

        
# testdata.py
# from employee import Employee
# from access_control import AccessControl

EMPLOYEES = {
    1: Employee(1,'bob','4 Jul 1994',80000.00),
    2: Employee(2,'carol','20 May 1992',85000.00),
    3: Employee(3,'ted','13 Feb 1998',55000.00),
    4: Employee(4,'alice','27 Nov 1981',40000.00),
    101: Employee(101,'morgan the manager','15 Dec 1972',120000.00)
}

ACCESSCONTROL = {
    101: AccessControl(101,True)
}

# __main__.py
# from testdata import EMPLOYEES,ACCESSCONTROL

def get_employee_info(empids,reqid):
    for empid in empids:
        if empid not in EMPLOYEES:
            continue
        employee = EMPLOYEES[empid]
        details = "Employee id %d, Name %s"
        details = details % (employee.empid,employee.name)
        
        if reqid in ACCESSCONTROL:
            if ACCESSCONTROL[reqid].can_see_personal:
                details += (", Birthdate: %s, Salary: %f" %(employee.birthday,employee.salary))

        print(details)        


# takes 2 args , list of empid to print and requester id

get_employee_info([3,4],3) # requestor may NOT see personal data

get_employee_info([1,2],101) # requestor may see personal data

Employee id 3, Name ted
Employee id 4, Name alice
Employee id 1, Name bob, Birthdate: 4 Jul 1994, Salary: 80000.000000
Employee id 2, Name carol, Birthdate: 20 May 1992, Salary: 85000.000000


In [8]:
# problem : add new access controls, add new attributes to employees
# solution : separate the concerns of the main program, which is retrieving and printing employee data
# from the data access itself, which requires access control checking
# how? use proxy pattern
# we'll use protection proxy in this case

In [9]:
## Proxy pattern
# classified as structual pattern.
# acts on a real subject, that would be the employees in the demo
# proxy keeps a reference to the subject
# exposes an identical interface to the client, the client is unaware that it is using a proxy
# proxy controls access to the real subject, and may even be resposible for creating and deleting it

<img src="uml/proxy_pattern.png">

In [11]:
### Proxy pattern structure

## AbsSubject class
# the subject represents the object that needs to be controlled in some way
# request() method is whatever operation the client needs to get access
# there can be multiple operations/requests

## ConcreteSubject class
# implements the abstract subject(AbsSubject class)

## Proxy class
# abstract subject is also implemented by the proxy, which will control access to the subject
# request () method and ConcreteSubject attribute
# contains reference to ConcreteSubject
# this reference is used when client program makes a request ConcreteSubject.request()
# the proxy controls the request, even to the point of denying the request all together, and returns the results

# Client ; unaware that a proxy is being used
# it only knows that it is talking to an abstract interface, that supports the methods in the abstract subject

Another way to implement this pattern using inheritance

<img src="uml/proxy_pattern_inheritance.png">

In [13]:
# client program directly interfaces with the proxy
# proxy inherits from ConcreteSubject
# but we want to follow composition over inheritance

In [1]:
# implement the protection proxy for employees example
# create 3 classes
# Abstract base class for subject ie employees collection, the request method will be the get_employee method we had before
# Concrete subject class, which is the employees collection itself, implements the abstract base class and actually retrieves the data
# Proxy subject class, 

# in the main program use a factory to get the proxy

In [4]:
# employee.py

class Employee:
    
    def __init__(self,empid,name,birthday,salary):
        self.empid = empid
        self.name = name
        self.birthday = birthday
        self.salary = salary # restrict access to birthdate and salary of the employee object


# access_control.py

class AccessControl:
    
    def __init__(self,empid,can_see_personal):
        self.empid = empid
        self.can_see_personal = can_see_personal # Bool if true allow request from that empid to see personal info


# to-do implement access_controls
# access_controls.py
# class AccessControls
        
        
# testdata.py
# from employee import Employee
# from access_control import AccessControl

EMPLOYEES = {
    1: Employee(1,'bob','4 Jul 1994',80000.00),
    2: Employee(2,'carol','20 May 1992',85000.00),
    3: Employee(3,'ted','13 Feb 1998',55000.00),
    4: Employee(4,'alice','27 Nov 1981',40000.00),
    101: Employee(101,'morgan the manager','15 Dec 1972',120000.00)
}

ACCESSCONTROL = {
    101: AccessControl(101,True)
}


# abs_employees.py
from abc import ABCMeta,abstractmethod

class AbsEmployees(metaclass=ABCMeta):
    
    @abstractmethod
    def get_employee_info(self,empids):
        pass
    
    
# employees.py
# from abs_employees import AbsEmployees
# from tessdata import EMPLOYEES

class Employees(AbsEmployees):
    
    def get_employee_info(self,empids):
        return (EMPLOYEES[empid] for empid in empids if empid in EMPLOYEES)

    
# proxy.py
# from abs_empoyees import AbsEmployees
# from employee import Employee
# from access_control import AccessControl

class Proxy(AbsEmployees):
    
    def __init__(self,employees,reqid):
        self._employees = employees
        self._reqid = reqid
    
    def get_employee_info(self,empids):
        reqid = self._reqid
        acc = AccessControls.get_access_control()
        
        for e in self._employees.get_employee_info(empids):
            if e.empid == reqid or \
                (reqid in acc and acc[reqid].can_see_personal):
                
                yield Employee(e.empid,e.name,e.salary,e.birthdate)
            else:
                yield Employee(e.empid,e.name,"*****","*****")
                
        
# factory.py
# from employees import Employees
# from proxy import Proxy

def get_employees_collection(reqid):
    return Proxy(Employees(),reqid)


# __main__.py
# from factory import get_employees_collection

DETAILS = "Employee Id: %d, Name: %s, Birthdate: %s, Salary: %f"

def print_employee_details(empids,reqid):
    employees = get_employees_collection(reqid)
    for e in employees.get_employee_info(empids):
        print(DETAILS % (e.empid,e.name,e.birthdate,e.salary))



print("Requestor authorized to see everything")
print_employee_details([1,2],101)


print("Ordinary employee, restricted access")
print_employee_details([1,2,3,4],3)

Requestor authorized to see everything


NameError: name 'AccessControls' is not defined

In [1]:
### Consequences
# introduces a level of indirection when accessing a real subject
# eg) protection proxy controls access
# virtual proxy can be used for lazy instantiation or to cache results
# eg) python functools.lru_cache
# remote proxy can hide communication details
# eg) pyodbc for database access, which could be on a remote server 
# smart proxy can add house keeping
# eg) use a proxy for locking, to access a data structure for a multi threaded application

## Summary
# When to use proxy pattern?
# if you want to add controls to an object

# SUMMARY

In [2]:
# 1) Facade Pattern

# class GetEmployeesFacade(AbsFacade):
#     def get_employees(self):
#         connection = pyodbc.connect(CONNSTR)
#         cursor = connection.cursor()
#         cursor.execute(QUERY)
#         for row in cursor:
#             print(row.FirstName,row.LastName)
#         connection.commit()
#         connection.close()
    
# facade pattern is a way to implement a common interface, to access a varity of APIs
# that have similar goals but differ in representation
# used this for access employee records via a db server

In [3]:
# 2) Adapter Pattern

# class VendAdapter(AbsAdapter):
    
#     @abstractproperty
#     def name(self):
#         return self.adaptee.name
    
#     @abstractproperty
#     def address(self):
#         return "{} {}".format(self.adaptee.number, self.adaptee.street)
    
# Converts an API into an interface the clients expect
# 2 ways to implement this pattern
# object adapters(uses composition and works with parent and sub classes both) and 
# class adapters (uses inheritance and only works with sub classes)
# using adapters let the client retrieve information for vendors and customers without knowing which one it was using

In [4]:
# 3) Decorator pattern

# class V6(AbsDecorator):
    
#     @property
#     def description(self):
#         return self.car.description + ", V6"
    
#     @property
#     def cost(self):
#         return self.car.cost + 1200.00

# provides a way to add responsibilities to an object without violating open closed or introducting sub classes
# eg) car dealership where options like engine tyoe , color etc were added using decorations without modifying the base class.

In [5]:
# 4) Template pattern

# class Airplane(AbsTransport):
# 
#     # necessary implementation of abstract method
#     def start_engine(self):
#         print("starting the gas turbine engine")
    
#     # over writes concrete method 
#     def leave_terminal(self):
#         print("Taking off")
    
#     # necessary implementation of abstract method
#     def travel_to_destination(self):
#         print("Flying")
    
#     # over writes hook method
#     def entertainment(self):
#         print("Playing in flight movie")
    
#     # over writes concrete method
#     def land_at_destination(self):
#         print("Landing at {}".format(self._destination))


# defines the basic outline of an algorithm,
# and then lets subclasses implement one or more steps while inforcing the order of steps vis template() method
# steps can be required, optional, default implementation or hooks
# eg) used to set up different kinds of transportation

In [6]:
# 5) Iterator pattern

# class Employees(Iterator):
#     _employees = {}
#     _headcount = 0
#     _empid = 0
    
#     def add_employee(self,employee):
#         self._headcount += 1
#         self._employees[self._headcount] = employee
    
#     def __iter__(self):
#         self._empid = 0
#         return self
    
#     def __next__(self):
#         if self._empid < self._headcount:
#             self._empid += 1
#             return self._employees[self._empid]
#         else:
#             return StopIteration


# add iteration over any collection
# that makes it easy to write for loops against a collection
# simple iterator: allows only one instance at a time
# generators: which provide lazy iterators
# multiple iterators : which allow multiple ACTIVE iterations

In [7]:
# 6) Composite pattern

# class Tree(Iterable,AbsComposite):
    
#     def __init__(self,members):
#         self.members = members
        
#     def __iter__(self):
#         return iter(self.members)
    
#     # takes pair of elements of the tree and returns the greater, this is recursive depth first traversal of tree
#     def get_oldest(self):
#         def f(t1,t2):
#             t1_,t2_ = t1.get_oldest() ,t2.get_oldest()
#             return t1_ if t1.birthdate < t2.birthdate else t2_
           
#         return reduce(f,self,NullPerson())


# provide single interface to objects in tree structures, both nodes and leaves
# violates the single resp, but the benifits out weight
# eg) used it to provide access to a family tree

In [8]:
# 7) State Pattern

# class ShoppingCart:
    
#     def __init__(self):
#         self.empty = Empty(self)
#         self.not_empty = NotEmpty(self)
#         self.check_out = AtCheckOut(self)
#         self.paid_for = PaidFor(self)
        
#         self.items = 0
#         self.state = self.empty


# eg) implemented shopping cart using state pattern
# build states and transitions, that are transparent to client code and easy to maintain

In [9]:
# 8) Proxy pattern

# class Proxy(AbsEmployees):
    
#     def __init__(self,employees,reqid):
#         self._employees = employees
#         self._reqid = reqid
    
#     def get_employee_info(self,empids):
#         reqid = self._reqid
#         acc = AccessControls.get_access_control()
        
#         for e in self._employees.get_employee_info(empids):
#             if e.empid == reqid or \
#                 (reqid in acc and acc[reqid].can_see_personal):
                
#                 yield Employee(e.empid,e.name,e.salary,e.birthdate)
#             else:
#                 yield Employee(e.empid,e.name,"*****","*****")


# a way to access control to objects
# 4 common types:
# g) protection proxy used to restrict access to employee info